In [1]:
import requests
import urllib.request
import json
import pandas as pd
import zipfile
import io

In [2]:
#Get download link from StatCan API
table_id = "14100287"
meta_url = f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{table_id}/en"
meta = requests.get(meta_url).json()
zip_url = meta['object']

In [3]:
#Download and read main CSV
response = requests.get(zip_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    print("ZIP contains:", z.namelist())
    for file_name in z.namelist():
        if file_name.endswith(".csv") and "MetaData" not in file_name:
            with z.open(file_name) as f:
                df_unemp = pd.read_csv(f, low_memory=False)

ZIP contains: ['14100287.csv', '14100287_MetaData.csv']


In [4]:
#Confirm it loaded
print("✅ Raw shape:", df_unemp.shape)
print("✅ Columns:", df_unemp.columns.tolist())
df_unemp.head()

✅ Raw shape: (5340654, 19)
✅ Columns: ['REF_DATE', 'GEO', 'DGUID', 'Labour force characteristics', 'Gender', 'Age group', 'Statistics', 'Data type', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']


,REF_DATE,GEO,DGUID,Labour force characteristics,Gender,Age group,Statistics,Data type,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1976-01,Canada,2021A000011124,Population,Total - Gender,15 years and over,Estimate,Seasonally adjusted,Persons in thousands,428,thousands,3,v2062809,1.1.1.1.1.1,16852.4,NaN,NaN,NaN,1
1,1976-01,Canada,2021A000011124,Population,Total - Gender,15 years and over,Estimate,Unadjusted,Persons in thousands,428,thousands,3,v2064888,1.1.1.1.1.2,16852.4,NaN,NaN,NaN,1
2,1976-01,Canada,2021A000011124,Population,Total - Gender,15 to 64 years,Estimate,Seasonally adjusted,Persons in thousands,428,thousands,3,v21580997,1.1.1.8.1.1,15015.9,NaN,NaN,NaN,1
3,1976-01,Canada,2021A000011124,Population,Total - Gender,15 to 64 years,Estimate,Unadjusted,Persons in thousands,428,thousands,3,v21580998,1.1.1.8.1.2,15015.9,NaN,NaN,NaN,1
4,1976-01,Canada,2021A000011124,Population,Total - Gender,15 to 24 years,Estimate,Seasonally adjusted,Persons in thousands,428,thousands,3,v2062836,1.1.1.2.1.1,4509.9,NaN,NaN,NaN,1


In [5]:
#Values we need
print(df_unemp["Labour force characteristics"].unique())


['Population' 'Labour force' 'Employment' 'Full-time employment'
 'Part-time employment' 'Unemployment' 'Unemployment rate'
 'Participation rate' 'Employment rate']


In [6]:
print(df_unemp["Gender"].unique())

['Total - Gender' 'Men+' 'Women+']


In [7]:
print(df_unemp["Age group"].unique())

['15 years and over' '15 to 64 years' '15 to 24 years' '15 to 19 years'
 '20 to 24 years' '25 years and over' '25 to 54 years' '55 years and over'
 '55 to 64 years']


In [8]:
#Filtering for needed and renamed columns
df_unemp = df_unemp[
    (df_unemp["Gender"] == "Total - Gender") &
    (df_unemp["Age group"] == "15 years and over") &
    (df_unemp["Labour force characteristics"] == "Unemployment rate")
]
df_unemp = df_unemp[["REF_DATE", "GEO", "VALUE"]]
df_unemp.rename(columns={"VALUE": "Unemployment_Rate"}, inplace=True)

print("Filtered shape:", df_unemp.shape)

Filtered shape: (32670, 3)


In [9]:
df_unemp.head()

,REF_DATE,GEO,Unemployment_Rate
636,1976-01,Canada,7.1
637,1976-01,Canada,8.1
638,1976-01,Canada,NaN
639,1976-01,Canada,NaN
640,1976-01,Canada,NaN


In [10]:
#CPI - Get API Link
table_id = "18100004"
meta_url = f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{table_id}/en"
meta = requests.get(meta_url).json()
zip_url = meta['object']

In [11]:
#Download CSV
response = requests.get(zip_url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    for file_name in z.namelist():
        if file_name.endswith(".csv") and "MetaData" not in file_name:
            with z.open(file_name) as f:
                df_cpi = pd.read_csv(f, low_memory=False)

In [12]:
#Columns
print(df_cpi.columns.tolist())

['REF_DATE', 'GEO', 'DGUID', 'Products and product groups', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']


In [13]:
df_cpi.head()

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
1,1914-01,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1
2,1914-01,Canada,2016A000011124,Goods and services,2002=100,17,units,0,v41691221,2.273,6.0,NaN,NaN,t,1
3,1914-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.200,6.0,NaN,NaN,NaN,1
4,1914-02,Canada,2016A000011124,All-items (1992=100),1992=100,7,units,0,v41713403,2.309,7.2,NaN,NaN,t,1


In [14]:
print(df_cpi["Products and product groups"].unique())

['All-items' 'All-items (1992=100)' 'Goods and services' 'Food'
 'Fresh or frozen poultry' 'Fresh or frozen chicken' 'Dairy products'
 'Fresh milk' 'Butter' 'Bakery and cereal products (excluding baby food)'
 'Breakfast cereal and other cereal products (excluding baby food)'
 'Flour and flour-based mixes' 'Fresh fruit' 'Apples' 'Oranges' 'Bananas'
 'Fresh vegetables' 'Potatoes' 'Tomatoes' 'Lettuce' 'Rented accommodation'
 'Rent' "Tenants' insurance premiums" 'Owned accommodation'
 'Mortgage interest cost' "Homeowners' replacement cost"
 'Property taxes and other special charges'
 "Homeowners' home and mortgage insurance"
 "Homeowners' maintenance and repairs" 'Water, fuel and electricity'
 'Electricity' 'Natural gas' 'Fuel oil and other fuels'
 'Telephone services' 'Postal and other communications services'
 'Furniture' 'Area rugs and mats' 'Clothing and footwear'
 'Dry cleaning services' 'Other clothing services' 'Transportation'
 'Private transportation' 'Purchase and leasing of pass

In [15]:
# Filter for "All-items" only
df_cpi_cleaned = df_cpi[df_cpi["Products and product groups"] == "All-items"]

# Focus on Canada (you can filter other GEOs if needed)
df_cpi_cleaned = df_cpi_cleaned[df_cpi_cleaned["GEO"] == "Canada"]

# Keep only relevant columns
df_cpi_cleaned = df_cpi_cleaned[["REF_DATE", "GEO", "VALUE"]]

# Rename for clarity
df_cpi_cleaned.rename(columns={"VALUE": "CPI"}, inplace=True)

# Convert REF_DATE to datetime for consistency with COVID dataset
df_cpi_cleaned["REF_DATE"] = pd.to_datetime(df_cpi_cleaned["REF_DATE"])

# Round values
df_cpi_cleaned["CPI"] = df_cpi_cleaned["CPI"].round(2)

In [16]:
df_cpi_cleaned.head()

,REF_DATE,GEO,CPI
0,1914-01-01,Canada,6.0
3,1914-02-01,Canada,6.0
6,1914-03-01,Canada,6.0
9,1914-04-01,Canada,6.0
12,1914-05-01,Canada,5.9


In [17]:
# Load the dataset for COVID
df_covid = pd.read_csv("daily-new-confirmed-covid-19-cases-per-million-people.csv")

In [18]:
df_covid.columns.tolist()

['Entity',
 'Day',
 'Daily new confirmed cases of COVID-19 per million people (rolling 7-day average, right-aligned)']

In [19]:
df_covid.head()

,Entity,Day,"Daily new confirmed cases of COVID-19 per million people (rolling 7-day average, right-aligned)"
0,Afghanistan,2020-01-09,0.0
1,Afghanistan,2020-01-10,0.0
2,Afghanistan,2020-01-11,0.0
3,Afghanistan,2020-01-12,0.0
4,Afghanistan,2020-01-13,0.0


In [27]:
#Filter for Canada only
df_covid_canada = df_covid[df_covid["Entity"] == "Canada"].copy()

In [25]:
#Rename columns for simplicity
df_covid_canada.rename(columns={
    "Day": "REF_DATE",
    "Daily new confirmed cases of COVID-19 per million people (rolling 7-day average, right-aligned)": "COVID_Cases_per_Million"
}, inplace=True)


In [ ]:
df_covid_canada.head()

In [ ]:
#Convert REF_DATE to datetime
df_covid_canada["REF_DATE"] = pd.to_datetime(df_covid_canada["REF_DATE"])

In [ ]:
#Keep only relevant columns
df_covid_canada = df_covid_canada[["REF_DATE", "COVID_Cases_per_Million"]]

In [ ]:
#Round for readability
df_covid_canada["COVID_Cases_per_Million"] = df_covid_canada["COVID_Cases_per_Million"].round(2)

In [ ]:
#Columns
df_covid_canada.columns.tolist()

In [ ]:
#Preview
df_covid_canada.head(10)

In [ ]:
# Convert REF_DATE or Day columns to datetime
df_unemp["REF_DATE"] = pd.to_datetime(df_unemp["REF_DATE"])
df_cpi_cleaned["REF_DATE"] = pd.to_datetime(df_cpi_cleaned["REF_DATE"])
df_covid_canada["REF_DATE"] = pd.to_datetime(df_covid_canada["REF_DATE"])

In [ ]:
df_covid_canada.head()

In [ ]:
# Start with CPI and Unemployment
df_merged = pd.merge(df_cpi_cleaned, df_unemp, on="REF_DATE", how="outer")

# Merge in COVID data
df_merged = pd.merge(df_merged, df_covid_canada, on="REF_DATE", how="outer")

# Sort by date
df_merged.sort_values("REF_DATE", inplace=True)

In [ ]:
df_merged.head(100)

In [ ]:
#Date adjustment from 2015 onwards
df_unemp["REF_DATE"] = pd.to_datetime(df_unemp["REF_DATE"])
df_cpi_cleaned["REF_DATE"] = pd.to_datetime(df_cpi_cleaned["REF_DATE"])
df_covid_canada["REF_DATE"] = pd.to_datetime(df_covid_canada["REF_DATE"])


In [ ]:
df_unemp = df_unemp[df_unemp["REF_DATE"].dt.year >= 2015]
df_cpi_cleaned = df_cpi_cleaned[df_cpi_cleaned["REF_DATE"].dt.year >= 2015]
df_covid_canada = df_covid_canada[df_covid_canada["REF_DATE"].dt.year >= 2015]


In [ ]:
df_unemp.reset_index(drop=True, inplace=True)
df_cpi_cleaned.reset_index(drop=True, inplace=True)
df_covid_canada.reset_index(drop=True, inplace=True)


In [ ]:
# Start with CPI and Unemployment
df_merged = pd.merge(df_cpi_cleaned, df_unemp, on="REF_DATE", how="outer")

# Merge in COVID data
df_merged = pd.merge(df_merged, df_covid_canada, on="REF_DATE", how="outer")

# Sort by date
df_merged.sort_values("REF_DATE", inplace=True)

In [ ]:
df_merged.head(10)

In [ ]:
df_merged.drop(columns=["GEO_x"], inplace=True)

In [ ]:
df_merged.head()

In [ ]:
df_merged.rename(columns={"GEO_y": "GEO"}, inplace=True)

In [ ]:
df_merged.head()

In [ ]:
#Fixing missing data issues 
print(df_merged["REF_DATE"].sort_values().unique())


In [ ]:
#Dates to monthly period format
df_merged["REF_DATE"] = pd.to_datetime(df_merged["REF_DATE"])
df_merged["REF_DATE"] = df_merged["REF_DATE"].dt.to_period("M").dt.to_timestamp()


In [ ]:
df_monthly = df_merged.groupby(["REF_DATE", "GEO"]).mean(numeric_only=True).reset_index()


In [ ]:
#For missing months
full_months = pd.date_range(start="2015-01-01", end="2025-12-01", freq="MS")
df_full = pd.DataFrame({"REF_DATE": full_months})

# Merge with your data, per GEO if needed
df_final = pd.merge(df_full, df_monthly, on="REF_DATE", how="left")


In [ ]:
df_final.head(10)

In [ ]:
df_merged.to_csv("Final_macro_covid_data.csv", index=False)